In [ ]:
from sentence_transformers import SentenceTransformer, models

## Language Model
transformer_model = models.Transformer('klue/roberta-base')

## Pooling Layer
pooling_layer = models.Pooling(transformer_model.get_word_embedding_dimension(), pooling_mode_mean_tokens=True)

## LM + Pooling
embedding_model = SentenceTransformer(modules=[transformer_model, pooling_layer])
print(embedding_model)

Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: RobertaModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
)


In [ ]:
from datasets import load_dataset

klue_sts_train = load_dataset("klue", "sts", split="train")
klue_sts_test = load_dataset("klue", "sts", split='validation')

Generating train split:   0%|          | 0/11668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/519 [00:00<?, ? examples/s]

{'guid': 'klue-sts-v1_train_00000', 'source': 'airbnb-rtt', 'sentence1': '숙소 위치는 찾기 쉽고 일반적인 한국의 반지하 숙소입니다.', 'sentence2': '숙박시설의 위치는 쉽게 찾을 수 있고 한국의 대표적인 반지하 숙박시설입니다.', 'labels': {'label': 3.7, 'real-label': 3.714285714285714, 'binary-label': 1}}


In [4]:
sample = klue_sts_train[0]

for key, value in sample.items():
    print(key)
    print(value, '\n')

guid
klue-sts-v1_train_00000 

source
airbnb-rtt 

sentence1
숙소 위치는 찾기 쉽고 일반적인 한국의 반지하 숙소입니다. 

sentence2
숙박시설의 위치는 쉽게 찾을 수 있고 한국의 대표적인 반지하 숙박시설입니다. 

labels
{'label': 3.7, 'real-label': 3.714285714285714, 'binary-label': 1} 



In [5]:
klue_sts_train = klue_sts_train.train_test_split(test_size=0.1, seed=42)
klue_sts_train, klue_sts_eval = klue_sts_train['train'], klue_sts_train['test']

In [6]:
from sentence_transformers import InputExample

def prepare_sts_examples(dataset):
    examples = []
    for data in dataset:
        examples.append(InputExample(texts=[data['sentence1'], data['sentence2']], label=data['labels']['label'] / 5.0))

    return examples


train_examples = prepare_sts_examples(klue_sts_train)
eval_examples = prepare_sts_examples(klue_sts_eval)
test_examples = prepare_sts_examples(klue_sts_test)

In [8]:
print(len(train_examples))

sample = train_examples[10]
print(sample)

10501
<InputExample> label: 0.0, texts: 프라하 자체가 안전한 도시이긴 하지만요.; 물론 저희가 추위를 많이 타긴 하지만요!


In [9]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)

In [10]:
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator

eval_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(eval_examples)
test_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(test_examples)

In [13]:
test_evaluator(embedding_model)

0.36460670798564826

In [14]:
from sentence_transformers import losses

epochs = 5
model_name = 'klue/roberta-base'
model_save_path = f"./output/training_sts_{model_name.replace('/', '-')}"
loss_func = losses.CosineSimilarityLoss(model=embedding_model)

embedding_model.fit(train_objectives=[(train_dataloader, loss_func)],
                    evaluator=eval_evaluator,
                    epochs=epochs,
                    evaluation_steps=1000,
                    warmup_steps=100,
                    output_path=model_save_path)

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Iteration:   0%|          | 0/657 [00:00<?, ?it/s]

Iteration:   0%|          | 0/657 [00:00<?, ?it/s]

Iteration:   0%|          | 0/657 [00:00<?, ?it/s]

Iteration:   0%|          | 0/657 [00:00<?, ?it/s]

Iteration:   0%|          | 0/657 [00:00<?, ?it/s]

In [15]:
trained_embedding_model = SentenceTransformer(model_save_path)
test_evaluator(trained_embedding_model)

0.8932227127266475

In [20]:
from datasets import load_dataset

klue_mrc_train = load_dataset('klue', 'mrc', split='train')
klue_mrc_test = load_dataset('klue', 'mrc', split='validation')

print(klue_mrc_train[0])

df_train = klue_mrc_train.to_pandas()
df_test = klue_mrc_test.to_pandas()

df_train = df_train[['title', 'question', 'context']]
df_test = df_test[['title', 'question', 'context']]

{'title': '제주도 장마 시작 … 중부는 이달 말부터', 'context': '올여름 장마가 17일 제주도에서 시작됐다. 서울 등 중부지방은 예년보다 사나흘 정도 늦은 이달 말께 장마가 시작될 전망이다.17일 기상청에 따르면 제주도 남쪽 먼바다에 있는 장마전선의 영향으로 이날 제주도 산간 및 내륙지역에 호우주의보가 내려지면서 곳곳에 100㎜에 육박하는 많은 비가 내렸다. 제주의 장마는 평년보다 2~3일, 지난해보다는 하루 일찍 시작됐다. 장마는 고온다습한 북태평양 기단과 한랭 습윤한 오호츠크해 기단이 만나 형성되는 장마전선에서 내리는 비를 뜻한다.장마전선은 18일 제주도 먼 남쪽 해상으로 내려갔다가 20일께 다시 북상해 전남 남해안까지 영향을 줄 것으로 보인다. 이에 따라 20~21일 남부지방에도 예년보다 사흘 정도 장마가 일찍 찾아올 전망이다. 그러나 장마전선을 밀어올리는 북태평양 고기압 세력이 약해 서울 등 중부지방은 평년보다 사나흘가량 늦은 이달 말부터 장마가 시작될 것이라는 게 기상청의 설명이다. 장마전선은 이후 한 달가량 한반도 중남부를 오르내리며 곳곳에 비를 뿌릴 전망이다. 최근 30년간 평균치에 따르면 중부지방의 장마 시작일은 6월24~25일이었으며 장마기간은 32일, 강수일수는 17.2일이었다.기상청은 올해 장마기간의 평균 강수량이 350~400㎜로 평년과 비슷하거나 적을 것으로 내다봤다. 브라질 월드컵 한국과 러시아의 경기가 열리는 18일 오전 서울은 대체로 구름이 많이 끼지만 비는 오지 않을 것으로 예상돼 거리 응원에는 지장이 없을 전망이다.', 'news_category': '종합', 'source': 'hankyung', 'guid': 'klue-mrc-v1_train_12759', 'is_impossible': False, 'question_type': 1, 'question': '북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은?', 'answers': {'answer_start': [478, 478], 'text'

In [22]:
def add_ir_context(df):
    irrelevant_contexts = []
    for idx, row in df.iterrows():
        title = row['title']
        irrelevant_contexts.append(df.query(f"title != '{title}'").sample(n=1)['context'].values[0])
    df['irrelevant_context'] = irrelevant_contexts
    return df
    
df_train_ir = add_ir_context(df_train)
df_test_ir = add_ir_context(df_test)

In [27]:
sample = df_train_ir.iloc[0]
print(f"title\n{sample['title']}\n")
print(f"question\n{sample['question']}\n")
print(f"context\n{sample['context']}\n")
print(f"irrelevant_context\n{sample['irrelevant_context']}\n")

title
제주도 장마 시작 … 중부는 이달 말부터

question
북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은?

context
올여름 장마가 17일 제주도에서 시작됐다. 서울 등 중부지방은 예년보다 사나흘 정도 늦은 이달 말께 장마가 시작될 전망이다.17일 기상청에 따르면 제주도 남쪽 먼바다에 있는 장마전선의 영향으로 이날 제주도 산간 및 내륙지역에 호우주의보가 내려지면서 곳곳에 100㎜에 육박하는 많은 비가 내렸다. 제주의 장마는 평년보다 2~3일, 지난해보다는 하루 일찍 시작됐다. 장마는 고온다습한 북태평양 기단과 한랭 습윤한 오호츠크해 기단이 만나 형성되는 장마전선에서 내리는 비를 뜻한다.장마전선은 18일 제주도 먼 남쪽 해상으로 내려갔다가 20일께 다시 북상해 전남 남해안까지 영향을 줄 것으로 보인다. 이에 따라 20~21일 남부지방에도 예년보다 사흘 정도 장마가 일찍 찾아올 전망이다. 그러나 장마전선을 밀어올리는 북태평양 고기압 세력이 약해 서울 등 중부지방은 평년보다 사나흘가량 늦은 이달 말부터 장마가 시작될 것이라는 게 기상청의 설명이다. 장마전선은 이후 한 달가량 한반도 중남부를 오르내리며 곳곳에 비를 뿌릴 전망이다. 최근 30년간 평균치에 따르면 중부지방의 장마 시작일은 6월24~25일이었으며 장마기간은 32일, 강수일수는 17.2일이었다.기상청은 올해 장마기간의 평균 강수량이 350~400㎜로 평년과 비슷하거나 적을 것으로 내다봤다. 브라질 월드컵 한국과 러시아의 경기가 열리는 18일 오전 서울은 대체로 구름이 많이 끼지만 비는 오지 않을 것으로 예상돼 거리 응원에는 지장이 없을 전망이다.

irrelevant_context
에드빈 피셔(Edwin Fischer, 1886년 10월 6일 ~ 1960년 1월 24일)는 스위스에서 태어나 독일에서 활동한 피아니스트이자 지휘자이다.

흔히 독일의 피아니스트라고 하나, 출생국은 스위스이다. 독일에서 주로 활약하고 바흐, 모차르트, 슈베르트의 작품에서 순수하게 독일적이고 고전적인

In [28]:
from sentence_transformers import InputExample

examples = []
for idx, row in df_test_ir.iterrows():
    examples.append(InputExample(texts=[row['question'], row['context']], 
                                 label=1))
    examples.append(InputExample(texts=[row['question'], row['irrelevant_context']],
                                 label=0))

In [29]:
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator

evaluator = EmbeddingSimilarityEvaluator.from_input_examples(examples)
evaluator(trained_embedding_model)

0.826860779505915

In [31]:
from sentence_transformers import datasets

train_samples = []
for idx, row in df_train_ir.iterrows():
    train_samples.append(InputExample(texts=[row['question'], row['context']]))

batch_size = 16
loader = datasets.NoDuplicatesDataLoader(train_samples, batch_size=batch_size)

In [32]:
from sentence_transformers import losses

loss = losses.MultipleNegativesRankingLoss(trained_embedding_model)

In [33]:
epochs = 5
save_path = "./output/training_mrc_klue-roberta-base"

trained_embedding_model.fit(train_objectives=[(loader, loss)],
                            epochs=epochs,
                            warmup_steps=100,
                            output_path=save_path,
                            show_progress_bar=True)

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1097 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1097 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1097 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1097 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1097 [00:00<?, ?it/s]

In [34]:
evaluator(trained_embedding_model)

0.8599035510257997

In [35]:
from sentence_transformers.cross_encoder import CrossEncoder

cross_model = CrossEncoder('klue/roberta-base', num_labels=1)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [36]:
from sentence_transformers.cross_encoder.evaluation import CECorrelationEvaluator

ce_evaluator = CECorrelationEvaluator.from_input_examples(examples)
ce_evaluator(cross_model)

0.020377511284339317

In [37]:
train_samples = []
for idx, row in df_train_ir.iterrows():
    train_samples.append(InputExample(texts=[row['question'], row['context']], label=1))
    train_samples.append(InputExample(texts=[row['question'], row['irrelevant_context']], label=0))

In [38]:
epochs = 5
batch_size = 16
model_save_path = "./output/training_mrc_cross-encoder-roberta-base"

train_dataloader = DataLoader(train_samples, shuffle=True, batch_size=batch_size)

cross_model.fit(train_dataloader=train_dataloader, epochs=epochs, warmup_steps=100, output_path=model_save_path)

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Iteration:   0%|          | 0/2195 [00:00<?, ?it/s]

Iteration:   0%|          | 0/2195 [00:00<?, ?it/s]

Iteration:   0%|          | 0/2195 [00:00<?, ?it/s]

Iteration:   0%|          | 0/2195 [00:00<?, ?it/s]

Iteration:   0%|          | 0/2195 [00:00<?, ?it/s]

In [39]:
ce_evaluator(cross_model)

0.8652622896663202

In [40]:
from datasets import load_dataset

klue_mrc_test = load_dataset('klue', 'mrc', split='validation')
klue_mrc_test = klue_mrc_test.train_test_split(test_size=1000, seed=42)['test']

In [41]:
import faiss

def make_embedding_index(model, corpus):
    embeddings = model.encode(corpus)
    index = faiss.IndexFlatL2(embeddings.shape[1])
    index.add(embeddings)

    return index

def find_embedding_top_k(query, model, index, k):
    query_embedding = model.encode([query])
    distances, indices = index.search(query_embedding, k)

    return indices

In [42]:
import numpy as np

def make_question_context_pairs(question_idx, indices):
    return [[klue_mrc_test['question'][question_idx], klue_mrc_test['context'][idx]] for idx in indices]

def rerank_top_k(cross_model, question_idx, indices, k):
    input_examples = make_question_context_pairs(question_idx, indices)
    relevance_scores = cross_model.predict(input_examples)
    reranked_indices = indices[np.argsort(relevance_scores)[::-1]]

    return reranked_indices

In [ ]:
# import time

# def evaluate_hit_rate(datasets, embedding_model, index, k=10):
#     ## bi-encoder only
#     start_time = time.time()

#     predictions = []
#     for question in datasets['question']:
#         predictions.append(find_embedding_top_k(question, embedding_model, index, k)[0])

#     total_prediction_count = len(predictions)
#     hit_count = 0
#     questions = datasets['question']
#     contexts = datasets['context']
#     for idx, prediction in enumerate(predictions):
#         for pred in prediction:
#           if contexts[pred] == contexts[idx]:
#             hit_count += 1
#             break
    
#     end_time = time.time()
#     return hit_count / total_prediction_count, end_time - start_time

In [ ]:
# finetuned_embedding_model = SentenceTransformer('shangrilar/klue-roberta-base-klue-sts-mrc')
# finetuned_index = make_embedding_index(finetuned_embedding_model, klue_mrc_test['context'])
# evaluate_hit_rate(klue_mrc_test, finetuned_embedding_model, finetuned_index, 10)
# # (0.946, 14.309881687164307)

In [45]:
import time
import numpy as np
from tqdm.auto import tqdm

def evaluate_hit_rate_with_rerank(datasets, embedding_model, cross_model, index, bi_k=30, cross_k=10):
    start_time = time.time()

    predictions = []
    for question_idx, question in enumerate(tqdm(datasets['question'])):
        indices = find_embedding_top_k(question, embedding_model, index, bi_k)[0] ## bi-encoder
        predictions.append(rerank_top_k(cross_model, question_idx, indices, k=cross_k)) ## cross-encoder
  
    total_prediction_count = len(predictions)
    hit_count = 0
    questions = datasets['question']
    contexts = datasets['context']
    for idx, prediction in enumerate(predictions):
        for pred in prediction:
            if contexts[pred] == contexts[idx]:
                hit_count += 1
                break
    end_time = time.time()
    return hit_count / total_prediction_count, end_time - start_time, predictions

In [46]:
index = make_embedding_index(trained_embedding_model, klue_mrc_test['context'])
hit_rate, cosumed_time, predictions = evaluate_hit_rate_with_rerank(klue_mrc_test, trained_embedding_model, cross_model, index, bi_k=30, cross_k=10)
hit_rate, cosumed_time
# (0.973, 1103.055629491806)

  0%|          | 0/1000 [00:00<?, ?it/s]

(0.978, 287.2874445915222)